In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)

data = pd.DataFrame()

pivot_data_table = pd.DataFrame()
data_stats_table = pd.DataFrame()

address = './CSV_FILES/GfK_POS_Tracking-Smart_Audio_Home_Systems_DE_&_UK_python.csv'

monthsOrdered = ['16-Dec',
          '17-Jan', '17-Feb', '17-Mar', '17-Apr', '17-May', '17-Jun',
          '17-Jul', '17-Aug', '17-Sep', '17-Oct', '17-Nov', '17-Dec',
          '18-Jan', '18-Feb', '18-Mar', '18-Apr', '18-May','18-Jun',
          '18-Jul', '18-Aug', '18-Sep', '18-Oct', '18-Nov']



columnsThatNeedATypeChange = ['sales_units', 'sales_value_eur', 'price_eur', 'sales_value_usd', 'price_usd']

columnsToBeSortedBy = ['id','channel' ,'year','period']

unique_ids =[]

sales_columns =  ['sales_units', 'sales_value_eur', 'price_eur', 'sales_value_usd', 'price_usd']

measurements_columns = ['height_in_mm','width_in_mm']

key_stats_columns = [ 'id', 'country', 'channel', 'year', 'period', 'height_in_mm', 'width_in_mm','sales_units', 'sales_value_eur', 'price_eur', 'sales_value_usd', 'price_usd']

pivot_key_columns = ['brand', 'model', 'id',
                     'smart_connect', 'smart_h._ecosys', 'operating_ai', 'airplay',
                     'googlecast/home', 'bluetooth', 'ethernet', 'wifi', 'height_in_mm',
                     'width_in_mm', 'high-res_audio', 'type_of_docking', 'multiroom',
                     'no.speakerboxes', 'output_channel', 'streaming_connection',
                     'streaming_technology', 'usb_connection', 'wattage_total']


def set_column_names(data):
    setColumnNames = ['COUNTRY','CHANNEL' ,'BRAND','MODEL', 'ID','YEAR','PERIOD','SMART CONNECT','SMART H. ECOSYS',
                      'OPERATING AI','AIRPLAY','GOOGLECAST/HOME','BLUETOOTH','ETHERNET','WIFI','HEIGHT IN MM','WIDTH IN MM','HIGH-RES AUDIO',
                      'TYPE OF DOCKING','MULTIROOM','NO.SPEAKERBOXES','OUTPUT CHANNEL','Streaming Connection','Streaming Technology',
                      'USB CONNECTION','WATTAGE TOTAL','Sales Units','Sales Value EUR','Price EUR','Sales Value USD','Price USD']
    data.columns = setColumnNames
    return data.columns

def set_column_names_to_lower(data):
    return data.columns.str.replace(' ', '_').str.lower()

def set_column_period_to_ordered_months(data):
    return data.period.astype('category',categories=monthsOrdered)

def set_to_numeric_for_these_columns(data,cols):
    for col in cols:
        data[col] = data[col].str.replace(',','').str.lower()
        data[col] = data[col].apply(pd.to_numeric, errors='coerce')
    return data

def set_to_numeric_for_mesurement_columns(data,cols):
    for col in cols:
        data[col] = data[col].apply(pd.to_numeric, errors='coerce')
    return data

def sort_values_based_on_these_columns(data,cols):
    data = data.sort_values(cols,inplace=False)
    return data

def sanitize_df(data):
    data.columns = set_column_names(data)
    data.columns = set_column_names_to_lower(data)
    data.period = set_column_period_to_ordered_months(data)
    data = set_to_numeric_for_these_columns(data,columnsThatNeedATypeChange)
    data = set_to_numeric_for_mesurement_columns(data,measurements_columns)
    data = sort_values_based_on_these_columns(data,columnsToBeSortedBy)
    return data

def get_pivot_and_stats_data_tables(data):
    pivot_data_table = data.filter(items=pivot_key_columns)
    data_stats_table = data.filter(items=key_stats_columns)
    return pivot_data_table,data_stats_table

def read_csv():
    data = pd.read_csv(address,low_memory=False)
    return data

def find_unique_of_this_column(data,col):
    return data[col].unique()

def get_describe_characterisitcs_for_sales_columns(firstRow,df,cols):
    for col in cols:
        firstRow = give_describe_characterisitcs(firstRow,df,col)
    return firstRow

def give_describe_characterisitcs(firstRow,df,col):
    sumCol = col + '_sum'
    sumCol = col + '_sum'
    meanCol = col + '_mean'
    medianCol = col + '_median'
    modeCol = col + '_mode'
    stdCol = col + '_std'
    minCol = col + '_min'
    maxCol = col + '_max'
    firstRow[sumCol]  = df[col].sum()
    firstRow[meanCol] = df[col].mean()
    firstRow[medianCol] = df[col].median()
    firstRow[modeCol] = df[col].mode()
    firstRow[stdCol] = df[col].std()
    firstRow[minCol] = df[col].min()
    firstRow[maxCol] = df[col].max()
    return firstRow

def get_describe_characterisitcs_for_measurement_columns(firstRow,df,cols):
    dfWithOutHeightAndWidth = df.dropna(subset=['height_in_mm','width_in_mm'],how='any')
    for col in cols:
        firstRow = give_describe_characterisitcs(firstRow,dfWithOutHeightAndWidth,col)
    return firstRow

def sort_by_id(df):
    i = 1
    dfBasedOnID = pd.DataFrame()
    for uid in unique_ids:
        allDataBasedOnThisID = df[df['id']==uid]
        try:
            firstRow = allDataBasedOnThisID.iloc[0]
            try:
                firstRow = get_describe_characterisitcs_for_sales_columns(firstRow,allDataBasedOnThisID,sales_columns)
                try:
                    dfBasedOnID = dfBasedOnID.append(firstRow, ignore_index=True)
                except:
                    print('cant append to dataframe')    
            except:
                print('cant return first row for sales columns')
        except:
            print('no Id')
        i += 1
        print(i)
        if(i > 20):
            break
            
    return dfBasedOnID



def main():
    '''
    send list of files to be parsed
    '''
    global unique_ids
    global data
    global pivot_data_table, data_stats_table
    data = read_csv()
    data = sanitize_df(data)
    
    pivot_data_table,data_stats_table = get_pivot_and_stats_data_tables(data)
    
    unique_ids = find_unique_of_this_column(data,'id')
    
#     create_specific_dfs(data)
    
    
    

if __name__ == "__main__": main()



/Users/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


In [2]:
data

country                     channel                     brand  \
5188        Germany  Consumer Electronic Stores                     NOXON   
5006        Germany  Consumer Electronic Stores                     NOXON   
4807        Germany  Consumer Electronic Stores                     NOXON   
4608        Germany  Consumer Electronic Stores                     NOXON   
4408        Germany  Consumer Electronic Stores                     NOXON   
3760        Germany  Consumer Electronic Stores                     NOXON   
3576        Germany  Consumer Electronic Stores                     NOXON   
3501        Germany  Consumer Electronic Stores                     NOXON   
3204        Germany  Consumer Electronic Stores                     NOXON   
2993        Germany  Consumer Electronic Stores                     NOXON   
2718        Germany  Consumer Electronic Stores                     NOXON   
2466        Germany  Consumer Electronic Stores                     NOXON   
2251        Germany  Consumer Electronic Stores                     NOXON   
1996        Germany  Consumer Electronic Stores                     NOXON   
1796        Germany  Consumer Electronic Stores                     NOXON   
1545        Germany  Consumer Electronic Stores                     NOXON   
1361        Germany  Consumer Electronic Stores                     NOXON   
980         Germany  Consumer Electronic Stores                     NOXON   
5148        Germany    Mass Merchandisers/DIYSs                     NOXON   
4953        Germany    Mass Merchandisers/DIYSs                     NOXON   
4789        Germany    Mass Merchandisers/DIYSs                     NOXON   
4561        Germany    Mass Merchandisers/DIYSs                     NOXON   
4423        Germany    Mass Merchandisers/DIYSs                     NOXON   
4201        Germany    Mass Merchandisers/DIYSs                     NOXON   
4002        Germany    Mass Merchandisers/DIYSs                     NOXON   
3855        Germany    Mass Merchandisers/DIYSs                     NOXON   
3621        Germany    Mass Merchandisers/DIYSs                     NOXON   
3468        Germany    Mass Merchandisers/DIYSs                     NOXON   
3236        Germany    Mass Merchandisers/DIYSs                     NOXON   
3027        Germany    Mass Merchandisers/DIYSs                     NOXON   
2778        Germany    Mass Merchandisers/DIYSs                     NOXON   
2557        Germany    Mass Merchandisers/DIYSs                     NOXON   
2374        Germany    Mass Merchandisers/DIYSs                     NOXON   
2173        Germany    Mass Merchandisers/DIYSs                     NOXON   
1705        Germany    Mass Merchandisers/DIYSs                     NOXON   
1209        Germany    Mass Merchandisers/DIYSs                     NOXON   
1007        Germany    Mass Merchandisers/DIYSs                     NOXON   
742         Germany    Mass Merchandisers/DIYSs                     NOXON   
474         Germany    Mass Merchandisers/DIYSs                     NOXON   
251         Germany    Mass Merchandisers/DIYSs                     NOXON   
5050        Germany  Consumer Electronic Stores                 PANASONIC   
8218  Great Britain  Consumer Electronic Stores                 PANASONIC   
4860        Germany  Consumer Electronic Stores                 PANASONIC   
8091  Great Britain  Consumer Electronic Stores                 PANASONIC   
4659        Germany  Consumer Electronic Stores                 PANASONIC   
7991  Great Britain  Consumer Electronic Stores                 PANASONIC   
4476        Germany  Consumer Electronic Stores                 PANASONIC   
7877  Great Britain  Consumer Electronic Stores                 PANASONIC   
4315        Germany  Consumer Electronic Stores                 PANASONIC   
7770  Great Britain  Consumer Electronic Stores                 PANASONIC   
4112        Germany  Consumer Electronic Stores                 PANASONIC   
7650  Gr

In [3]:
data.describe()

year  height_in_mm  width_in_mm  output_channel    sales_units  \
count  8333.000000   7924.000000  7998.000000     8333.000000    8333.000000   
mean   2017.468139    159.423536   238.359502        1.630385     612.138606   
std       0.572283     89.807738   143.765331        0.776329    4316.129717   
min    2016.000000     15.000000    51.900000        0.000000       1.000000   
25%    2017.000000     95.000000   130.000000        2.000000       5.000000   
50%    2018.000000    161.450000   210.000000        2.000000      18.000000   
75%    2018.000000    203.000000   314.000000        2.000000      90.000000   
max    2018.000000   1540.000000  1000.000000        2.000000  187435.000000   

       sales_value_eur    price_eur  sales_value_usd    price_usd  
count     8.333000e+03  8333.000000     8.333000e+03  8333.000000  
mean      9.094023e+04   376.626785     1.047171e+05   435.539182  
std       3.689659e+05   486.275162     4.218234e+05   570.397806  
min       1.000000e+00     1.000000     1.000000e+00     1.000000  
25%       1.249000e+03   156.000000     1.453000e+03   178.000000  
50%       5.794000e+03   261.000000     6.666000e+03   301.000000  
75%       2.499000e+04   440.000000     2.884100e+04   503.000000  
max       9.989807e+06  5593.000000     1.061617e+07  6667.000000

In [4]:
data.dtypes

country                   object
channel                   object
brand                     object
model                     object
id                        object
year                       int64
period                  category
smart_connect             object
smart_h._ecosys           object
operating_ai              object
airplay                   object
googlecast/home           object
bluetooth                 object
ethernet                  object
wifi                      object
height_in_mm             float64
width_in_mm              float64
high-res_audio            object
type_of_docking           object
multiroom                 object
no.speakerboxes           object
output_channel             int64
streaming_connection      object
streaming_technology      object
usb_connection            object
wattage_total             object
sales_units                int64
sales_value_eur            int64
price_eur                  int64
sales_value_usd            int64
price_usd 

In [5]:
def create_df_by_unique_id_only():
    pass

    


In [6]:
def create_specific_dfs(data):
    pass

In [7]:
pivot_data_table

brand                 model           id  \
5188                     NOXON                NOVA S    100293423   
5006                     NOXON                NOVA S    100293423   
4807                     NOXON                NOVA S    100293423   
4608                     NOXON                NOVA S    100293423   
4408                     NOXON                NOVA S    100293423   
3760                     NOXON                NOVA S    100293423   
3576                     NOXON                NOVA S    100293423   
3501                     NOXON                NOVA S    100293423   
3204                     NOXON                NOVA S    100293423   
2993                     NOXON                NOVA S    100293423   
2718                     NOXON                NOVA S    100293423   
2466                     NOXON                NOVA S    100293423   
2251                     NOXON                NOVA S    100293423   
1996                     NOXON                NOVA S    100293423   
1796                     NOXON                NOVA S    100293423   
1545                     NOXON                NOVA S    100293423   
1361                     NOXON                NOVA S    100293423   
980                      NOXON                NOVA S    100293423   
5148                     NOXON                NOVA S    100293423   
4953                     NOXON                NOVA S    100293423   
4789                     NOXON                NOVA S    100293423   
4561                     NOXON                NOVA S    100293423   
4423                     NOXON                NOVA S    100293423   
4201                     NOXON                NOVA S    100293423   
4002                     NOXON                NOVA S    100293423   
3855                     NOXON                NOVA S    100293423   
3621                     NOXON                NOVA S    100293423   
3468                     NOXON                NOVA S    100293423   
3236                     NOXON                NOVA S    100293423   
3027                     NOXON                NOVA S    100293423   
2778                     NOXON                NOVA S    100293423   
2557                     NOXON                NOVA S    100293423   
2374                     NOXON                NOVA S    100293423   
2173                     NOXON                NOVA S    100293423   
1705                     NOXON                NOVA S    100293423   
1209                     NOXON                NOVA S    100293423   
1007                     NOXON                NOVA S    100293423   
742                      NOXON                NOVA S    100293423   
474                      NOXON                NOVA S    100293423   
251                      NOXON                NOVA S    100293423   
5050                 PANASONIC               SC-ALL2    100671708   
8218                 PANASONIC               SC-ALL2    100671708   
4860                 PANASONIC               SC-ALL2    100671708   
8091                 PANASONIC               SC-ALL2    100671708   
4659                 PANASONIC               SC-ALL2    100671708   
7991                 PANASONIC               SC-ALL2    100671708   
4476                 PANASONIC               SC-ALL2    100671708   
7877                 PANASONIC               SC-ALL2    100671708   
4315                 PANASONIC               SC-ALL2    100671708   
7770                 PANASONIC               SC-ALL2    100671708   
4112                 PANASONIC               SC-ALL2    100671708   
7650                 PANASONIC               SC-ALL2    100671708   
3934                 PANASONIC               SC-ALL2    100671708   
7539                 PANASONIC               SC-ALL2    100671708   
3749                 PANASONIC               SC-ALL2    100671708   
7421                 PANASONIC               SC-ALL2    100671708   
3561                 PANASONIC               SC-ALL2    100671708   
7315                 PA

In [8]:
data_stats_table

id        country                     channel  year  period  \
5188    100293423        Germany  Consumer Electronic Stores  2016  16-Dec   
5006    100293423        Germany  Consumer Electronic Stores  2017  17-Jan   
4807    100293423        Germany  Consumer Electronic Stores  2017  17-Feb   
4608    100293423        Germany  Consumer Electronic Stores  2017  17-Mar   
4408    100293423        Germany  Consumer Electronic Stores  2017  17-Apr   
3760    100293423        Germany  Consumer Electronic Stores  2017  17-Jul   
3576    100293423        Germany  Consumer Electronic Stores  2017  17-Aug   
3501    100293423        Germany  Consumer Electronic Stores  2017  17-Sep   
3204    100293423        Germany  Consumer Electronic Stores  2017  17-Oct   
2993    100293423        Germany  Consumer Electronic Stores  2017  17-Nov   
2718    100293423        Germany  Consumer Electronic Stores  2017  17-Dec   
2466    100293423        Germany  Consumer Electronic Stores  2018  18-Jan   
2251    100293423        Germany  Consumer Electronic Stores  2018  18-Feb   
1996    100293423        Germany  Consumer Electronic Stores  2018  18-Mar   
1796    100293423        Germany  Consumer Electronic Stores  2018  18-Apr   
1545    100293423        Germany  Consumer Electronic Stores  2018  18-May   
1361    100293423        Germany  Consumer Electronic Stores  2018  18-Jun   
980     100293423        Germany  Consumer Electronic Stores  2018  18-Aug   
5148    100293423        Germany    Mass Merchandisers/DIYSs  2016  16-Dec   
4953    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-Jan   
4789    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-Feb   
4561    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-Mar   
4423    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-Apr   
4201    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-May   
4002    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-Jun   
3855    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-Jul   
3621    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-Aug   
3468    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-Sep   
3236    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-Oct   
3027    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-Nov   
2778    100293423        Germany    Mass Merchandisers/DIYSs  2017  17-Dec   
2557    100293423        Germany    Mass Merchandisers/DIYSs  2018  18-Jan   
2374    100293423        Germany    Mass Merchandisers/DIYSs  2018  18-Feb   
2173    100293423        Germany    Mass Merchandisers/DIYSs  2018  18-Mar   
1705    100293423        Germany    Mass Merchandisers/DIYSs  2018  18-May   
1209    100293423        Germany    Mass Merchandisers/DIYSs  2018  18-Jul   
1007    100293423        Germany    Mass Merchandisers/DIYSs  2018  18-Aug   
742     100293423        Germany    Mass Merchandisers/DIYSs  2018  18-Sep   
474     100293423        Germany    Mass Merchandisers/DIYSs  2018  18-Oct   
251     100293423        Germany    Mass Merchandisers/DIYSs  2018  18-Nov   
5050    100671708        Germany  Consumer Electronic Stores  2016  16-Dec   
8218    100671708  Great Britain  Consumer Electronic Stores  2016  16-Dec   
4860    100671708        Germany  Consumer Electronic Stores  2017  17-Jan   
8091    100671708  Great Britain  Consumer Electronic Stores  2017  17-Jan   
4659    100671708        Germany  Consumer Electronic Stores  2017  17-Feb   
7991    100671708  Great Britain  Consumer Electronic Stores  2017  17-Feb   
4476    100671708        Germany  Consumer Electronic Stores  2017  17-Mar   
7877    100671708  Great Britain  Consumer Electronic Stores  2017  17-Mar   
4315    100671708        Germany  Consumer Electronic Stores  2017  17-Apr   
7770    100671708  Great Britain  Consumer Electronic Stores  2017  17-Apr   
4112    100671708        Germany  Con

In [9]:
data.columns

Index(['country', 'channel', 'brand', 'model', 'id', 'year', 'period',
       'smart_connect', 'smart_h._ecosys', 'operating_ai', 'airplay',
       'googlecast/home', 'bluetooth', 'ethernet', 'wifi', 'height_in_mm',
       'width_in_mm', 'high-res_audio', 'type_of_docking', 'multiroom',
       'no.speakerboxes', 'output_channel', 'streaming_connection',
       'streaming_technology', 'usb_connection', 'wattage_total',
       'sales_units', 'sales_value_eur', 'price_eur', 'sales_value_usd',
       'price_usd'],
      dtype='object')

In [10]:
collectedID = sort_by_id(data_stats_table)

/Users/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mike/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:915: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
/Users/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:108: SettingWithCopyWarning: 
A

In [11]:
collectedID

channel  country  height_in_mm         id  period  \
0   Consumer Electronic Stores  Germany        140.00  100293423  16-Dec   
1   Consumer Electronic Stores  Germany        174.00  100671708  16-Dec   
2   Consumer Electronic Stores  Germany        207.00  101024462  16-Dec   
3   Consumer Electronic Stores  Germany        232.00  101024494  16-Dec   
4   Consumer Electronic Stores  Germany        175.00  101240478  16-Dec   
5   Consumer Electronic Stores  Germany        161.45  102215785  16-Dec   
6   Consumer Electronic Stores  Germany        189.00  102345694  16-Dec   
7   Consumer Electronic Stores  Germany        422.00  102402338  16-Dec   
8   Consumer Electronic Stores  Germany        100.00  104594624  16-Dec   
9   Consumer Electronic Stores  Germany         76.00  104664708  17-Jan   
10  Consumer Electronic Stores  Germany        115.00  104980264  16-Dec   
11  Consumer Electronic Stores  Germany         46.00  105107104  16-Dec   
12  Consumer Electronic Stores  Germany         70.00  105112279  16-Dec   
13  Consumer Electronic Stores  Germany        246.00  105227304  16-Dec   
14  Consumer Electronic Stores  Germany        212.00  105262570  16-Dec   
15  Consumer Electronic Stores  Germany        188.00  105262595  16-Dec   
16  Consumer Electronic Stores  Germany        172.00  105528291  16-Dec   
17  Consumer Electronic Stores  Germany        123.00  105568370  16-Dec   
18  Consumer Electronic Stores  Germany        166.00  105735826  16-Dec   
19  Consumer Electronic Stores  Germany        273.00  105735857  16-Dec   

    price_eur  price_eur_max  price_eur_mean  price_eur_median  price_eur_min  \
0       119.0          139.0       88.825000              91.5           57.0   
1       150.0          199.0      143.966667             150.5           42.0   
2       141.0          259.0      137.588235             137.0           86.0   
3       254.0          310.0      192.125000             190.0           92.0   
4       141.0          179.0      125.520000             126.0           86.0   
5       374.0          459.0      376.354167             396.0          252.0   
6       176.0          225.0       99.028571              85.5           42.0   
7       332.0          468.0      307.471264             307.0          146.0   
8       157.0          242.0      172.968750             193.5           85.0   
9       259.0          297.0      203.333333             211.0          110.0   
10      658.0          693.0      609.111111             612.5          475.0   
11      527.0          548.0      513.680000             518.0          449.0   
12      806.0          899.0      840.208333             853.5          644.0   
13      453.0          510.0      472.343750             472.0          417.0   
14      186.0          195.0      174.406250             175.5          149.0   
15      344.0          344.0      308.229167             306.5          265.0   
16      578.0          603.0      553.680000             563.0          430.0   
17      165.0          172.0      130.709677             134.0           67.0   
18      270.0          348.0      236.065934             240.0           25.0   
19      202.0          288.0      195.747126             189.0           65.0   

                                       price_eur_mode  price_eur_std  \
0                               0    100
dtype: int64      22.465800   
1                               0    150
dtype: int64      32.490431   
2   0     97
1    117
2    141
3    146
4    149
5...      34.942479   
3                      0    170
1    194
dtype: int64      53.094408   
4                      0    117
1    145
dtype: int64      22.579711   
5                      0    400
1    405
dtype: int64      50.179037   
6                                0    63
dtype: int64      41.299532   
7             0    262
1    279
2    325
dtype: int64      74.081011   
8                               0    199
dtype: int64      47.100773   
9   

In [12]:
collectedID.columns

Index(['channel', 'country', 'height_in_mm', 'id', 'period', 'price_eur',
       'price_eur_max', 'price_eur_mean', 'price_eur_median', 'price_eur_min',
       'price_eur_mode', 'price_eur_std', 'price_eur_sum', 'price_usd',
       'price_usd_max', 'price_usd_mean', 'price_usd_median', 'price_usd_min',
       'price_usd_mode', 'price_usd_std', 'price_usd_sum', 'sales_units',
       'sales_units_max', 'sales_units_mean', 'sales_units_median',
       'sales_units_min', 'sales_units_mode', 'sales_units_std',
       'sales_units_sum', 'sales_value_eur', 'sales_value_eur_max',
       'sales_value_eur_mean', 'sales_value_eur_median', 'sales_value_eur_min',
       'sales_value_eur_mode', 'sales_value_eur_std', 'sales_value_eur_sum',
       'sales_value_usd', 'sales_value_usd_max', 'sales_value_usd_mean',
       'sales_value_usd_median', 'sales_value_usd_min', 'sales_value_usd_mode',
       'sales_value_usd_std', 'sales_value_usd_sum', 'width_in_mm', 'year'],
      dtype='object')

In [22]:
len(unique_ids)

295